In [2]:
import os

os.environ['GRB_LICENSE_FILE'] = '/Users/k5cheng/gurobi.lic'

import gurobipy as gr

print(gr.gurobi.version())

license_path = os.getenv('GRB_LICENSE_FILE')
print("License file path:", license_path)

(12, 0, 2)
License file path: /Users/k5cheng/gurobi.lic


# 0. Dataset Analysis

In [3]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

def dataset_analysis(cap_file='cap.pck', pref_file='pref.pck', verbose=True):
    # load data
    with open(cap_file, 'rb') as f:
        capacities = pickle.load(f)
    with open(pref_file, 'rb') as f:
        preferences = pickle.load(f)

    if verbose:
        print("=== BASIC DATASET STATISTICS ===")
    school_ids = list(capacities.keys())
    all_ids = set(preferences.keys())
    student_ids = [id for id in all_ids if id not in capacities]
    
    if verbose:
        print(f"School tracks: {len(school_ids)}")
        print(f"Students: {len(student_ids)}")
        print(f"Total capacity: {sum(capacities.values())}")
        print(f"Capacity/student ratio: {sum(capacities.values())/len(student_ids):.2f}")
    
    if verbose:
        print("\n=== SCHOOL STRUCTURE ANALYSIS ===")
    # school suffixes (PRI, REG, PIE)
    suffixes = [s.split('_')[-1] if '_' in s else "UNKNOWN" for s in school_ids]
    suffix_counter = Counter(suffixes)
    if verbose:
        print(f"Track types: {dict(suffix_counter)}")
    
    # organize schools by base name (removing suffix)
    base_schools = defaultdict(list)
    for school_id in school_ids:
        if '_' in school_id:
            parts = school_id.split('_')
            suffix = parts[-1]
            base_name = '_'.join(parts[:-1])
            base_schools[base_name].append((school_id, suffix))
    
    if verbose:
        print(f"Unique base schools: {len(base_schools)}")
    
    # count tracks per base school
    track_counts = Counter([len(tracks) for _, tracks in base_schools.items()])
    if verbose:
        print(f"Tracks per base school: {dict(track_counts)}")
    
    # schools with PIE tracks
    pie_schools = [base for base, tracks in base_schools.items() 
                  if any(suffix == 'PIE' for _, suffix in tracks)]
    if verbose:
        print(f"\nSchools with PIE tracks ({len(pie_schools)}):")
    for base in pie_schools:
        tracks = base_schools[base]
        if verbose:
            print(f"  {base}: {[suffix for _, suffix in tracks]}")
        for school_id, suffix in tracks:
            if verbose:
                print(f"    {suffix}: capacity {capacities[school_id]}")

    if verbose:
        print("\n=== SCHOOL CAPACITY DISTRIBUTION ===")
    pri_capacities = [cap for s, cap in capacities.items() if s.endswith('_PRI')]
    reg_capacities = [cap for s, cap in capacities.items() if s.endswith('_REG')]
    pie_capacities = [cap for s, cap in capacities.items() if s.endswith('_PIE')]

    if verbose:
        print(f"PRI tracks: {len(pri_capacities)}, total capacity: {sum(pri_capacities)}")
        print(f"REG tracks: {len(reg_capacities)}, total capacity: {sum(reg_capacities)}")
        print(f"PIE tracks: {len(pie_capacities)}, total capacity: {sum(pie_capacities)}")
    
    if verbose:
        print("\n=== STUDENT PREFERENCE ANALYSIS ===")
    student_prefs = {s: preferences[s] for s in student_ids}
    
    # count preferences per student
    pref_counts = [len(prefs) for s, prefs in student_prefs.items()]
    if verbose:
        print(f"Avg preferences per student: {np.mean(pref_counts):.1f}")
        print(f"Min: {min(pref_counts)}, Max: {max(pref_counts)}")
    
    # students' preference distribution across the schools
    all_preferences = []
    for s, prefs in student_prefs.items():
        for _, school in prefs.items():
            all_preferences.append(school)
    
    school_popularity = Counter(all_preferences)
    
    # most popular schools
    if verbose:
        print("\nTop 20 most popular schools (across all preference ranks):")
    for school, count in school_popularity.most_common(20):
        capacity = capacities.get(school, 0)
        if verbose:
            print(f"{school}: {count} students ranked it (capacity: {capacity}, ratio: {count/capacity:.1f}x)")
    
    # preference distribution by rank
    preferences_by_rank = defaultdict(Counter)
    for s, prefs in student_prefs.items():
        for rank, school in prefs.items():
            preferences_by_rank[int(rank)][school] += 1
    
    # most first-choice schools, count preference/capcity ratio
    if verbose:
        print("\nTop 10 first-choice schools:")
    for school, count in preferences_by_rank[1].most_common(10):
        capacity = capacities.get(school, 0)
        if verbose:
            print(f"{school}: {count} first choices (capacity: {capacity}, ratio: {count/capacity:.1f}x)")
    
    if verbose:
        print("\n=== SCHOOL PRIORITY ANALYSIS ===")
    school_priorities = {s: preferences[s] for s in school_ids if s in preferences}
    
    # school's priority list length
    priority_counts = [len(priorities) for s, priorities in school_priorities.items()]
    if verbose:
        print(f"Avg students in priority list: {np.mean(priority_counts):.1f}")
        print(f"Min: {min(priority_counts)}, Max: {max(priority_counts)}")
    
    # check for empty priority lists
    empty_priority = [s for s, priorities in school_priorities.items() if len(priorities) == 0]
    if verbose:
        print(f"Schools with empty priority lists: {len(empty_priority)}")
    if empty_priority:
        if verbose:
            print(f"Examples: {empty_priority[:5]}")
    
    # compare same school (PRI vs REG) priority lists
    same_priorities = 0
    different_priorities = 0
    
    for base, tracks in base_schools.items():
        track_ids = [id for id, _ in tracks]
        priority_lists = {id: school_priorities.get(id, {}) for id in track_ids if id in school_priorities}
        
        if len(priority_lists) < 2:
            continue
        
        # compare all pairs of priority lists
        track_ids = list(priority_lists.keys())
        all_same = True
        
        for i in range(len(track_ids)):
            for j in range(i+1, len(track_ids)):
                list1 = list(priority_lists[track_ids[i]].values())
                list2 = list(priority_lists[track_ids[j]].values())
                
                if list1 != list2:
                    all_same = False
                    break
        
        if all_same:
            same_priorities += 1
        else:
            different_priorities += 1
    
    if verbose:
        print(f"Schools with identical priority lists across tracks: {same_priorities}")
        print(f"Schools with different priority lists across tracks: {different_priorities}")
    
    if verbose:
        print("\n=== POTENTIAL STABILITY ISSUES ===")
    
    # demand-supply ratio
    demand_supply = {}
    for school in school_ids:
        demand = school_popularity.get(school, 0)
        supply = capacities.get(school, 0)
        if supply > 0:
            ratio = demand / supply
            demand_supply[school] = ratio
    
    extreme_schools = [(s, ratio) for s, ratio in demand_supply.items() if ratio > 10]
    extreme_schools.sort(key=lambda x: x[1], reverse=True)
    
    if verbose:
        print(f"Schools with extreme demand-supply ratio (>10x):")
    for school, ratio in extreme_schools[:10]:
        demand = school_popularity.get(school, 0)
        supply = capacities.get(school, 0)
        if verbose:
            print(f"{school}: {demand} students prefer, capacity {supply}, ratio {ratio:.1f}x")
    

    # how many students have unique first choices
    first_choices = [next(iter(prefs.values())) for s, prefs in student_prefs.items() if prefs]
    unique_first = len(set(first_choices))
    total_first = len(first_choices)
    if verbose:
        print(f"\nStudents with unique first choices: {unique_first}/{total_first} ({unique_first/total_first*100:.1f}%)")


    # calculate preference correlation 
    
    
    # find valid student-school pairs
    valid_pairs = []
    for student_id, prefs in student_prefs.items():
        if isinstance(prefs, dict):
            for rank, school_id in prefs.items():
                if school_id in school_priorities:
                    if student_id in school_priorities[school_id].values():
                        valid_pairs.append((student_id, school_id))
    if verbose:
        print(f"\nNumber of valid (student, school) pairs: {len(valid_pairs)}")


    # for further use
    return {
        'student_ids': student_ids,
        'school_ids': school_ids,
        'school_capacities': capacities,
        'preferences': preferences,
        'student_preferences': student_prefs,
        'school_priorities': school_priorities,
        'base_schools': base_schools,
        'demand_supply': demand_supply,
        'preferences_by_rank': preferences_by_rank,
        'extreme_schools': extreme_schools,
        'valid_pairs': valid_pairs
    }


In [4]:
dataset_analysis();

=== BASIC DATASET STATISTICS ===
School tracks: 100
Students: 1395
Total capacity: 1624
Capacity/student ratio: 1.16

=== SCHOOL STRUCTURE ANALYSIS ===
Track types: {'REG': 49, 'PRI': 49, 'PIE': 2}
Unique base schools: 49
Tracks per base school: {2: 47, 3: 2}

Schools with PIE tracks (2):
  8417_401000000133: ['REG', 'PIE', 'PRI']
    REG: capacity 10
    PIE: capacity 2
    PRI: capacity 3
  8437_401000000133: ['REG', 'PIE', 'PRI']
    REG: capacity 23
    PIE: capacity 2
    PRI: capacity 5

=== SCHOOL CAPACITY DISTRIBUTION ===
PRI tracks: 49, total capacity: 268
REG tracks: 49, total capacity: 1352
PIE tracks: 2, total capacity: 4

=== STUDENT PREFERENCE ANALYSIS ===
Avg preferences per student: 6.5
Min: 2, Max: 32

Top 20 most popular schools (across all preference ranks):
24307_401000000232_PRI: 515 students ranked it (capacity: 10, ratio: 51.5x)
24307_401000000232_REG: 515 students ranked it (capacity: 54, ratio: 9.5x)
11680_401000000133_PRI: 368 students ranked it (capacity: 4, 

# 1. Baseline Model: Quadratic-Constrained

In [5]:
import gurobipy as gp
from gurobipy import GRB
import time
from collections import defaultdict

def create_quadratic_model(data, B, valid_pairs, max_seats_per_school=5):
    students = data['student_ids']
    schools = data['school_ids']
    capacities = data['school_capacities']
    student_preferences = data['student_preferences']
    school_priorities = data['school_priorities']
    
    # calculate preference weights
    w = {}
    for s in students:
        prefs = student_preferences.get(s, {})
        n = len(prefs)  
        for rank, c in prefs.items():
            if (s, c) in valid_pairs:
                w[s, c] = n - int(rank) + 1
    
    model = gp.Model("QuadraticStableMatching")
    
    # decision variables
    x = model.addVars(valid_pairs, vtype=GRB.BINARY, name="x")
    t = model.addVars(schools, vtype=GRB.INTEGER, lb=0, ub=max_seats_per_school, name="t")
    model._x = x
    model._t = t
    
    # objective function: maximize preference satisfaction
    model.setObjective(
        gp.quicksum(w[s, c] * x[s, c] for (s, c) in valid_pairs if (s, c) in w),
        GRB.MAXIMIZE  
    )

    # constraint 1: each student assigned to at most one school
    for s in students:
        model.addConstr(
            gp.quicksum(x[s, c] for (s2, c) in valid_pairs if s2 == s) <= 1,
            name=f"student_limit_{s}"
        )

    # constraint 2: school capacity
    for c in schools:
        model.addConstr(
            gp.quicksum(x[s, c2] for (s, c2) in valid_pairs if c2 == c) <= capacities[c] + t[c],
            name=f"capacity_{c}"
        )

    # constraint 3: total expansion limited by budget
    model.addConstr(
        gp.quicksum(t[c] for c in schools) <= B,
        name="total_expansion"
    )
    
    # constraint 4: stability
    for s in students:
        s_prefs = student_preferences.get(s, {})
        
        for s_rank, c in s_prefs.items():
            if (s, c) not in valid_pairs:
                continue
            
            # find schools that student s prefers more than c
            better_schools_for_s = [
                sch for r, sch in s_prefs.items() 
                if int(r) < int(s_rank) and (s, sch) in valid_pairs
            ]
            
            # find students that school c prioritizes over student s
            if c in school_priorities:
                c_priorities = school_priorities[c]
                s_pos = None
                for pos, student_id in c_priorities.items():
                    if student_id == s:
                        s_pos = int(pos)
                        break
                
                if s_pos is not None:
                    better_students_for_c = []
                    for pos, student_id in c_priorities.items():
                        if int(pos) < s_pos and (student_id, c) in valid_pairs:
                            better_students_for_c.append(student_id)
                    
                    if better_students_for_c:
                        # binary variable for stability condition
                        z = model.addVar(vtype=GRB.BINARY, name=f"z_{s}_{c}")
                        
                        # z = 1 if student s is not matched with c or a better school
                        model.addConstr(
                            x[s, c] + gp.quicksum(x[s, better] for better in better_schools_for_s) + z == 1,
                            name=f"z_def_{s}_{c}"
                        )
                        
                        # higher priority students assigned to c
                        higher_priority_sum = gp.quicksum(x[sp, c] for sp in better_students_for_c)
                        
                        # if z = 1, then higher priority students must fill capacity + expansion
                        # quadratic constraint: z * (capacities[c] + t[c]) <= higher_priority_sum
                        
                        # create a quadratic term for z * t[c]
                        model.addQConstr(
                            higher_priority_sum >= capacities[c] * z + z * t[c],
                            name=f"stability_{s}_{c}"
                        )
    
    return model

# 2. Linearization Model 1: Individualized Linearization

In [6]:
def create_individualized_linearization_model(data, B, valid_pairs, max_seats_per_school=5):
    students = data['student_ids']
    schools = data['school_ids']
    capacities = data['school_capacities']
    student_preferences = data['student_preferences']
    school_priorities = data['school_priorities']
    
    # calculate preference weights
    w = {}
    for s in students:
        prefs = student_preferences.get(s, {})
        n = len(prefs)  
        for rank, c in prefs.items():
            if (s, c) in valid_pairs:
                w[s, c] = n - int(rank) + 1
    
    model = gp.Model("IndividualizedLinearization")
    
    # decision variables
    x = model.addVars(valid_pairs, vtype=GRB.BINARY, name="x")
    t = model.addVars(schools, vtype=GRB.INTEGER, lb=0, ub=max_seats_per_school, name="t")
    model._x = x
    model._t = t
    
    # objective function: maximize preference satisfaction
    model.setObjective(
        gp.quicksum(w[s, c] * x[s, c] for (s, c) in valid_pairs if (s, c) in w),
        GRB.MAXIMIZE  
    )

    # constraint 1: each student assigned to at most one school
    for s in students:
        model.addConstr(
            gp.quicksum(x[s, c] for (s2, c) in valid_pairs if s2 == s) <= 1,
            name=f"student_limit_{s}"
        )

    # constraint 2: school capacity
    for c in schools:
        model.addConstr(
            gp.quicksum(x[s, c2] for (s, c2) in valid_pairs if c2 == c) <= capacities[c] + t[c],
            name=f"capacity_{c}"
        )

    # constraint 3: total expansion limited by budget
    model.addConstr(
        gp.quicksum(t[c] for c in schools) <= B,
        name="total_expansion"
    )
    
    # constraint 4: stability
    for s in students:
        s_prefs = student_preferences.get(s, {})
        
        for s_rank, c in s_prefs.items():
            if (s, c) not in valid_pairs:
                continue
            
            # find schools that student s prefers more than c
            better_schools_for_s = [
                sch for r, sch in s_prefs.items() 
                if int(r) < int(s_rank) and (s, sch) in valid_pairs
            ]
            
            # find students that school c prioritizes over student s
            if c in school_priorities:
                c_priorities = school_priorities[c]
                s_pos = None
                for pos, student_id in c_priorities.items():
                    if student_id == s:
                        s_pos = int(pos)
                        break
                
                if s_pos is not None:
                    better_students_for_c = []
                    for pos, student_id in c_priorities.items():
                        if int(pos) < s_pos and (student_id, c) in valid_pairs:
                            better_students_for_c.append(student_id)
                    
                    if better_students_for_c:
                        # binary variable for stability condition
                        z = model.addVar(vtype=GRB.BINARY, name=f"z_{s}_{c}")
                        
                        # z = 1 if student s is not matched with c or a better school
                        model.addConstr(
                            x[s, c] + gp.quicksum(x[s, better] for better in better_schools_for_s) + z == 1,
                            name=f"z_def_{s}_{c}"
                        )
                        
                        # higher priority students assigned to c
                        higher_priority_sum = gp.quicksum(x[sp, c] for sp in better_students_for_c)
                        
                        # create auxiliary variable for z * t[c]
                        alpha = model.addVar(
                            vtype=GRB.INTEGER, lb=0, ub=max_seats_per_school,
                            name=f"alpha_{s}_{c}"
                        )
                        
                        # McCormick-style linearization for z * t[c]
                        model.addConstr(
                            alpha <= max_seats_per_school * z,
                            name=f"alpha_bound1_{s}_{c}"
                        )
                        model.addConstr(
                            alpha <= t[c],
                            name=f"alpha_bound2_{s}_{c}"
                        )
                        model.addConstr(
                            alpha >= t[c] - max_seats_per_school * (1 - z),
                            name=f"alpha_bound3_{s}_{c}"
                        )
                        model.addConstr(
                            alpha >= 0,
                            name=f"alpha_bound4_{s}_{c}"
                        )
                        
                        model.addConstr(
                            higher_priority_sum >= capacities[c] * z + alpha,
                            name=f"stability_{s}_{c}"
                        )
    
    return model

# 3. Linearization Model 2: Aggregated Linearization 

In [7]:
def create_aggregated_linearization_model(data, B, valid_pairs, max_seats_per_school=5):
    students = data['student_ids']
    schools = data['school_ids']
    capacities = data['school_capacities']
    student_preferences = data['student_preferences']
    school_priorities = data['school_priorities']
    
    # calculate preference weights
    w = {}
    for s in students:
        prefs = student_preferences.get(s, {})
        n = len(prefs)  
        for rank, c in prefs.items():
            if (s, c) in valid_pairs:
                w[s, c] = n - int(rank) + 1
    
    model = gp.Model("AggregatedLinearization")
    
    # decision variables
    x = model.addVars(valid_pairs, vtype=GRB.BINARY, name="x")
    t = model.addVars(schools, vtype=GRB.INTEGER, lb=0, ub=max_seats_per_school, name="t")
    model._x = x
    model._t = t
    
    # objective function: maximize preference satisfaction
    model.setObjective(
        gp.quicksum(w[s, c] * x[s, c] for (s, c) in valid_pairs if (s, c) in w),
        GRB.MAXIMIZE  
    )

    # constraint 1: each student assigned to at most one school
    for s in students:
        model.addConstr(
            gp.quicksum(x[s, c] for (s2, c) in valid_pairs if s2 == s) <= 1,
            name=f"student_limit_{s}"
        )

    # constraint 2: school capacity
    for c in schools:
        model.addConstr(
            gp.quicksum(x[s, c2] for (s, c2) in valid_pairs if c2 == c) <= capacities[c] + t[c],
            name=f"capacity_{c}"
        )

    # constraint 3: total expansion limited by budget
    model.addConstr(
        gp.quicksum(t[c] for c in schools) <= B,
        name="total_expansion"
    )

    # group students by school and priority
    school_priority_groups = {}
    for c in schools:
        if c in school_priorities:
            # create a dictionary mapping priority to list of students
            priority_map = defaultdict(list)
            
            for pos, student_id in school_priorities[c].items():
                if (student_id, c) in valid_pairs:
                    priority_map[int(pos)].append(student_id)
            
            school_priority_groups[c] = priority_map
    
    # aggregated: create expansion variables at the priority group level
    priority_expansion = {}
    
    # constraint 4: stability
    for c in schools:
        if c not in school_priority_groups:
            continue
            
        priority_groups = school_priority_groups[c]
        priority_levels = sorted(priority_groups.keys())
        
        # one expansion variable per priority level for this school
        priority_expansion[c] = {}
        for priority in priority_levels:
            priority_expansion[c][priority] = model.addVar(
                vtype=GRB.INTEGER, lb=0, ub=max_seats_per_school,
                name=f"priority_expansion_{c}_{priority}"
            )
            
            # link to school expansion (cannot exceed total expansion)
            model.addConstr(
                priority_expansion[c][priority] <= t[c],
                name=f"expansion_bound_{c}_{priority}"
            )
        
        # for each priority level, handle all students at that level
        for i, priority in enumerate(priority_levels):
            students_at_priority = priority_groups[priority]
            
            # get all higher priority students for this school
            higher_priority_students = []
            for p in priority_levels[:i]:
                higher_priority_students.extend(priority_groups[p])
            
            if not higher_priority_students:
                continue
            
            # compute sum of higher priority students assigned to this school
            higher_priority_sum = gp.quicksum(x[sp, c] for sp in higher_priority_students)
            
            # for each student at this priority level
            for s in students_at_priority:
                s_prefs = student_preferences.get(s, {})
                
                # find position of c in s's preferences
                c_rank = None
                for rank, school in s_prefs.items():
                    if school == c:
                        c_rank = int(rank)
                        break
                
                if c_rank is None:
                    continue
                
                # find schools s prefers more than c
                better_schools_for_s = [
                    sch for r, sch in s_prefs.items() 
                    if int(r) < c_rank and (s, sch) in valid_pairs
                ]
                
                # binary variable for whether stability needs to be checked
                y = model.addVar(vtype=GRB.BINARY, name=f"y_{s}_{c}")
                
                # y = 1 if student s is not matched with c or a better school
                model.addConstr(
                    x[s, c] + gp.quicksum(x[s, better] for better in better_schools_for_s) + y == 1,
                    name=f"y_def_{s}_{c}"
                )
                
                # create auxiliary variable for y * priority_expansion[c][priority]
                beta = model.addVar(
                    vtype=GRB.INTEGER, lb=0, ub=max_seats_per_school,
                    name=f"beta_{s}_{c}"
                )
                
                # linearize y * priority_expansion[c][priority] with McCormick constraints
                model.addConstr(
                    beta <= max_seats_per_school * y,
                    name=f"beta_bound1_{s}_{c}"
                )
                
                model.addConstr(
                    beta <= priority_expansion[c][priority],
                    name=f"beta_bound2_{s}_{c}"
                )
                
                model.addConstr(
                    beta >= priority_expansion[c][priority] - max_seats_per_school * (1 - y),
                    name=f"beta_bound3_{s}_{c}"
                )
                
                # can be tightened to capacity + max expansion
                M = capacities[c] + max_seats_per_school
                

                model.addConstr(
                    higher_priority_sum >= capacities[c] * y + beta - M * (1 - y),
                    name=f"stability_{s}_{c}"
                )
    
    return model

# Solver & Comparison

In [8]:
def solve_model(model, budget, model_name):
    start_time = time.time()
    
    # gurobi parameters
    model.Params.OutputFlag = 0  # =1: show solver output 
    model.Params.TimeLimit = 300  # time limit
    model.Params.MIPGap = 0.02  # 2% optimality gap
    model.Params.Threads = 4  

    model.optimize()
    solve_time = time.time() - start_time
    
    # results
    status = model.Status
    if status == GRB.OPTIMAL:
        print(f"{model_name} found optimal solution.")
    elif status == GRB.TIME_LIMIT:
        print(f"{model_name} reached time limit with gap {model.MIPGap:.2%}.")
    elif status == GRB.SUBOPTIMAL:
        print(f"{model_name} found suboptimal solution.")
    elif status == GRB.INFEASIBLE:
        print(f"{model_name} is infeasible.")
        return solve_time, float('inf'), None, None, 0
    elif status == GRB.UNBOUNDED:
        print(f"{model_name} is unbounded.")
        return solve_time, float('inf'), None, None, 0
    else:
        print(f"{model_name} ended with status code: {status}")
        return solve_time, float('inf'), None, None, 0

    x = model._x
    t = model._t
    assignments = [(s, c) for (s, c), var in x.items() if var.X > 0.5]
    expansions = {c: int(var.X) for c, var in t.items() if var.X > 0.5}
    # metrics
    obj_value = model.ObjVal
    mip_gap = model.MIPGap if model.SolCount > 0 else float('inf')
    
    print(f"\n=== {model_name} Result ===")
    print(f"Success pairs: {len(assignments)}")
    # print(f"Model size: {model.NumVars} variables, {model.NumConstrs} constraints")
    # print(f"Iterations: {model.IterCount}, Nodes: {model.NodeCount}")
    
    # show example assignments / expansion summary
    for s, c in assignments[:5]:
        print(f"Student {s} -> School {c}")
    total_expansion = sum(expansions.values())
    expanded_schools = len(expansions)
    print(f"\nExpansion used: {int(total_expansion)}/{budget}")
    print(f"Schools expanded: {expanded_schools}")
    if expanded_schools > 0:
        print("\nTop 5 expansions:")
        top_expansions = sorted(expansions.items(), key=lambda x: x[1], reverse=True)
        for c, exp in top_expansions[:5]:
            print(f"School {c}: +{int(exp)} seats")
    
    print(f"\nObjective Value: {obj_value:.4f}")
    
    return solve_time, mip_gap, assignments, expansions, obj_value





def compare_models(data, valid_pairs, budget_values, max_seats_per_school=5):
    results = {
        'quadratic': [],
        'individualized': [],
        'aggregated': []
    }
    
    for B in budget_values:
        print(f"\n==== Testing with Budget B = {B} ====")
        
        # 1.
        # print("\n--- BUILDING QUADRATIC MODEL ---")
        quadratic_model = create_quadratic_model(data, B, valid_pairs, max_seats_per_school)
        quad_time, quad_gap, quad_assignments, quad_expansions, quad_obj = solve_model(
            quadratic_model, B, "Quadratic Model"
        )
        
        # 2.
        # print("\n--- BUILDING INDIVIDUALIZED LINEARIZATION MODEL ---")
        ind_model = create_individualized_linearization_model(data, B, valid_pairs, max_seats_per_school)
        ind_time, ind_gap, ind_assignments, ind_expansions, ind_obj = solve_model(
            ind_model, B, "Individualized Linearization Model"
        )
        
        # 3.
        # print("\n--- BUILDING AGGREGATED LINEARIZATION MODEL ---")
        agg_model = create_aggregated_linearization_model(data, B, valid_pairs, max_seats_per_school)
        agg_time, agg_gap, agg_assignments, agg_expansions, agg_obj = solve_model(
            agg_model, B, "Aggregated Linearization Model"
        )
        
        results['quadratic'].append({
            'budget': B,
            'time': quad_time,
            'gap': quad_gap,
            'obj': quad_obj,
            'assignments': len(quad_assignments) if quad_assignments else 0,
        })
        
        results['individualized'].append({
            'budget': B,
            'time': ind_time,
            'gap': ind_gap,
            'obj': ind_obj,
            'assignments': len(ind_assignments) if ind_assignments else 0,
        })
        
        results['aggregated'].append({
            'budget': B,
            'time': agg_time,
            'gap': agg_gap,
            'obj': agg_obj,
            'assignments': len(agg_assignments) if agg_assignments else 0,
        })
    
    return results




def print_comparison_table(results, budget_values):
    print("\n===== SOLVER TIME COMPARISON (seconds) =====")
    print(f"{'Budget':<10} {'Quadratic':<15} {'Individualized':<15} {'Aggregated':<15}")
    print("-" * 55)
    
    for i, B in enumerate(budget_values):
        q_time = results['quadratic'][i]['time']
        ind_time = results['individualized'][i]['time']
        agg_time = results['aggregated'][i]['time']
        print(f"{B:<10} {q_time:<15.2f} {ind_time:<15.2f} {agg_time:<15.2f}")
    
    print("\n===== MIP GAP COMPARISON =====")
    print(f"{'Budget':<10} {'Quadratic':<15} {'Individualized':<15} {'Aggregated':<15}")
    print("-" * 55)
    
    for i, B in enumerate(budget_values):
        q_gap = results['quadratic'][i]['gap']
        ind_gap = results['individualized'][i]['gap']
        agg_gap = results['aggregated'][i]['gap']
        print(f"{B:<10} {q_gap:<15.4f} {ind_gap:<15.4f} {agg_gap:<15.4f}")
    
    print("\n===== OBJECTIVE VALUE COMPARISON =====")
    print(f"{'Budget':<10} {'Quadratic':<15} {'Individualized':<15} {'Aggregated':<15}")
    print("-" * 55)
    
    for i, B in enumerate(budget_values):
        q_obj = results['quadratic'][i]['obj'] if results['quadratic'][i]['obj'] else 0
        ind_obj = results['individualized'][i]['obj'] if results['individualized'][i]['obj'] else 0
        agg_obj = results['aggregated'][i]['obj'] if results['aggregated'][i]['obj'] else 0
        print(f"{B:<10} {q_obj:<15.1f} {ind_obj:<15.1f} {agg_obj:<15.1f}")
    
    print("\n===== MATCHED STUDENTS COMPARISON =====")
    print(f"{'Budget':<10} {'Quadratic':<15} {'Individualized':<15} {'Aggregated':<15}")
    print("-" * 55)
    
    for i, B in enumerate(budget_values):
        q_matches = results['quadratic'][i]['assignments']
        ind_matches = results['individualized'][i]['assignments']
        agg_matches = results['aggregated'][i]['assignments']
        print(f"{B:<10} {q_matches:<15} {ind_matches:<15} {agg_matches:<15}")
    
    # average times and speedups
    avg_q = sum(r['time'] for r in results['quadratic']) / len(budget_values)
    avg_ind = sum(r['time'] for r in results['individualized']) / len(budget_values)
    avg_agg = sum(r['time'] for r in results['aggregated']) / len(budget_values)
    
    print("\n===== SUMMARY =====")
    print(f"Average times: Quadratic={avg_q:.2f}s, Individualized={avg_ind:.2f}s, Aggregated={avg_agg:.2f}s")
    print(f"Speedup Individualized/Quadratic: {avg_q/avg_ind:.2f}x")
    print(f"Speedup Aggregated/Quadratic: {avg_q/avg_agg:.2f}x")
    

def run_comparison():
    data = dataset_analysis(verbose=False)
    valid_pairs = data['valid_pairs']

    budget_values = [0, 10, 25, 50, 75, 100]
    max_seats_per_school = 5    # per-school expansion limit
    
    results = compare_models(data, valid_pairs, budget_values, max_seats_per_school)
    print_comparison_table(results, budget_values)

if __name__ == "__main__":
    run_comparison()


==== Testing with Budget B = 0 ====
Set parameter Username
Set parameter LicenseID to value 2664832
Academic license - for non-commercial use only - expires 2026-05-13
Quadratic Model found optimal solution.

=== Quadratic Model Result ===
Success pairs: 1220
Student 5b8630fc093da070f20eac50 -> School 8475_401000000132_REG
Student 5b86313e093da070f20fdf7f -> School 24307_401000000232_PRI
Student 5b863164093da070f2108e89 -> School 11678_401000000132_REG
Student 5b863134093da070f20fb091 -> School 24327_401000000133_REG
Student 5b8630e9093da070f20e551f -> School 8441_401000000133_PRI

Expansion used: 0/0
Schools expanded: 0

Objective Value: 5988.0000
Individualized Linearization Model found optimal solution.

=== Individualized Linearization Model Result ===
Success pairs: 1221
Student 5b8630fc093da070f20eac50 -> School 8475_401000000132_REG
Student 5b86313e093da070f20fdf7f -> School 24307_401000000232_PRI
Student 5b863164093da070f2108e89 -> School 11678_401000000132_REG
Student 5b86313